In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import shutil
import time
import tensorflow as tf
from tensorflow import keras
# import keras

print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)
matplotlib 3.0.3
numpy 1.16.2
pandas 0.24.2
sklearn 0.21.1
tensorflow 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [3]:
# x = (x - u) / std

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# x_train: [None, 28, 28] -> [None, 784]
x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)

In [4]:
# tf.keras.models.Sequential()

"""
model = tf.keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))
"""


model = tf.keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# relu: y = max(0, x)
# softmax: 将向量变成概率分布. x = [x1, x2, x3],
#          y = [e^x1/sum, e^x2/sum, e^x3/sum], sum = e^x1 + e^x2 + e^x3

# reason for sparse: y->index. y->one_hot->[]
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [5]:
# TensorBoard, EarlyStopping, ModelCheckpoint
log_dir = './tmp/callbacks'
if os.path.exists(log_dir):
    shutil.rmtree(log_dir)
    os.mkdir(log_dir)
else:
    os.mkdir(log_dir)
    
output_model_file = os.path.join(log_dir, 'fashion_mnist_model.h5')
callbacks = [
    keras.callbacks.TensorBoard(log_dir),
    keras.callbacks.ModelCheckpoint(output_model_file, save_best_only=True),
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3)
]
history = model.fit(x_train_scaled, y_train, epochs=10, 
                    validation_data=[x_valid_scaled, y_valid], 
                    callbacks=callbacks)

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 3s 50us/sample - loss: 0.9261 - accuracy: 0.7017 - val_loss: 0.6315 - val_accuracy: 0.7874
Epoch 2/10
55000/55000 [==============================] - 3s 50us/sample - loss: 0.5846 - accuracy: 0.7976 - val_loss: 0.5363 - val_accuracy: 0.8120
Epoch 3/10
55000/55000 [==============================] - 3s 50us/sample - loss: 0.5163 - accuracy: 0.8211 - val_loss: 0.4890 - val_accuracy: 0.8312
Epoch 4/10
55000/55000 [==============================] - 3s 49us/sample - loss: 0.4792 - accuracy: 0.8330 - val_loss: 0.4637 - val_accuracy: 0.8408
Epoch 5/10
55000/55000 [==============================] - 3s 48us/sample - loss: 0.4543 - accuracy: 0.8411 - val_loss: 0.4441 - val_accuracy: 0.8482
Epoch 6/10
55000/55000 [==============================] - 3s 49us/sample - loss: 0.4362 - accuracy: 0.8480 - val_loss: 0.4283 - val_accuracy: 0.8536
Epoch 7/10
55000/55000 [==============================] -